In [1]:
import mne
import numpy as np
import pandas as pd
import keras
import datasets
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn import metrics
from tensorflow import keras
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, SimpleRNN, Conv1D, TextVectorization
from keras.utils import to_categorical
from keras import regularizers, layers, Input, Model
from tensorflow.keras.optimizers import AdamW, SGD
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
import torch
import evaluate

/opt/anaconda3/envs/eeg2text/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from huggingface_hub import login
login()

In [2]:
# val_ds = load_dataset("groshanr/EEG2Text", split=[f"train[{k}%:{k+10}%]" for k in range(0, 100, 10)])
# train_ds = load_dataset("groshanr/EEG2Text", split=[f"train[:{k}%]+train[{k+10}%:]" for k in range(0, 100, 10)])

ds = load_dataset("groshanr/EEG2Text", split = 'train')

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
# epochs = mne.read_epochs("Murphy et al Dataset/session0_files0-151-epo/rsvp_session0_files0-11-epo.fif")
# eeg_data = epochs.get_data() # NumPy array of shape [n_trials, n_electrodes, n_timepoints] with n_electrodes = 64, n_timepoints = 276 and n_trials is variable by each file.
# metadata = epochs.metadata   # Pandas DataFrame

In [4]:
print(ds)

Dataset({
    features: ['time', 'condition', 'Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz', 'trigger', 'word', 'pos', 'sess', 'sent_num', 'filename', 'len', 'freq', 'sess_id', 'prev_pos', 'next_pos', 'prev_freq', 'next_freq', 'prev_len', 'next_len', 'sent_ident', '__index_level_0__'],
    num_rows: 111560580
})


: 

In [ ]:
df = ds.to_pandas()

In [ ]:
df.head()

In [36]:
df['pos'] = df['pos'].astype('category')
df['pos'] = df['pos'].cat.codes

In [ ]:
df = df.sample(frac=0.25) # Subsample

In [37]:
# Create train/test split
X = df[['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 
                'CP5', 'CP1', 'Pz', 'P3', 'P7','O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 
                'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 
                'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 
                'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 
                'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz']].values
   

y = df['pos'].values

In [38]:
kf = KFold(n_splits=10)

In [45]:
# rnn_filepath = 'best_rnn.keras'
# rnn_checkpoint = ModelCheckpoint(filepath=rnn_filepath, 
#                              monitor='val_loss',
#                              verbose=1, 
#                              save_best_only=True,
#                              mode='min')
# rnn_callbacks = [rnn_checkpoint]

In [ ]:
# lstm_filepath = 'best_lstm.keras'
# lstm_checkpoint = ModelCheckpoint(filepath=rnn_filepath,
#                              monitor='val_loss',
#                              verbose=1,
#                              save_best_only=True,
#                              mode='min')
# lstm_callbacks = [lstm_checkpoint]

### Simple NN Test

In [ ]:
for train_index, test_index in kf.split(X, y):
    x_train = X[train_index]
    y_train = y[train_index]
    x_test = X[test_index]
    y_test = y[test_index]
    y_train = to_categorical(y_train, num_classes=16)
    y_test = to_categorical(y_test, num_classes=16)

    model_rnn= Sequential()
    model_rnn.add(Dense(256, input_dim = x_train.shape[1]))
    model_rnn.add(Dense(256))
    model_rnn.add(Dense(256))
    model_rnn.add(Dense(16, activation='softmax'))
    model_rnn.summary()

# Train the model
    opt = AdamW(learning_rate=0.0001)
    model_rnn.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    trained_model_base = model_rnn.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

In [ ]:
from sklearn.metrics import classification_report
y_pred = model_rnn.predict(x_test)
y_pred_m = y_pred.reshape(y_pred.shape[0], y_pred.shape[2])
y_test_m = y_test.reshape(y_test.shape[0], y_test.shape[2])
y_pred_m = np.argmax(y_pred_m, axis=1)
y_test_m = np.argmax(y_test_m, axis=1)
print(classification_report(y_test_m, y_pred_m))



### LSTM Test

In [ ]:
# for train_index, test_index in kf.split(X, y):
#     x_train = X[train_index]
#     y_train = y[train_index]
#     x_test = X[test_index]
#     y_test = y[test_index]
#     y_train = to_categorical(y_train)
#     y_test = to_categorical(y_test)
#     x_train = x_train.reshape(x_train.shape[0], -1, x_train.shape[1])
#     x_test = x_test.reshape(x_test.shape[0], -1, x_test.shape[1])
#     y_train = y_train.reshape(y_train.shape[0], -1, y_train.shape[1])
#     y_test = y_test.reshape(y_test.shape[0], -1, y_test.shape[1])
#     model_lstm = Sequential()
#     model_lstm.add(layers.LSTM(64, return_sequences=True, input_shape=(x_train.shape[1],x_train.shape[2])))
#     model_lstm.add(layers.LSTM(64, return_sequences=True))
#     model_lstm.add(layers.LSTM(64, return_sequences=True))
#     model_lstm.add(Dense(y_train.shape[2], activation='softmax'))
#     model_lstm.summary()

#     # Train the model
#     opt = AdamW(learning_rate=0.001)
#     model_lstm.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
#     trained_model_base = model_lstm.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

In [ ]:
# from sklearn.metrics import classification_report
# y_pred = model_lstm.predict(x_test)
# y_pred_m = y_pred.reshape(y_pred.shape[0], y_pred.shape[2])
# y_test_m = y_test.reshape(y_test.shape[0], y_test.shape[2])
# y_pred_m = np.argmax(y_pred_m, axis=1)
# y_test_m = np.argmax(y_test_m, axis=1)
# print(classification_report(y_test_m, y_pred_m))
